In [30]:
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os 
import random

In [6]:
!pip install sklearn

  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2951 sha256=0c560fd4848649d958951063490c797a60349baf8ab9df8f0095a90d6df19e31
  Stored in directory: /home/zhanglangeba/.cache/pip/wheels/5c/47/e1/386fa7740f1947ad87d16c68041c16ad8c23ca2bfd139a3fe5
Successfully built sklearn


In [10]:
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'sklearn'

In [31]:
def load_data():
    datafile="housing.data"
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_lens=len(feature_names)
    data= data.reshape(-1,feature_lens)
    ratio=0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    maximums,minimums=training_data.max(axis=0),training_data.min(axis=0)  #不应该是1  列算吗
    global max_values
    global min_values
   
    max_values = maximums
    min_values = minimums
    
    # 对数据进行归一化处理
    for i in range(feature_lens):
        data[:, i] = (data[:, i] - min_values[i]) / (maximums[i] - minimums[i])
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

In [32]:
training_data, test_data = load_data()
print(training_data.shape)
print(training_data[1,:])

(404, 14)
[2.35922547e-04 0.00000000e+00 2.62405723e-01 0.00000000e+00
 1.72839552e-01 5.47997713e-01 7.82698274e-01 3.48961979e-01
 4.34782617e-02 1.14822544e-01 5.53191364e-01 1.00000000e+00
 2.04470202e-01 3.68888885e-01]


In [33]:
class Reg(paddle.nn.Layer):
    def __init__(self):
        super(Reg,self).__init__()
        self.Net=Linear(in_features=13,out_features=1)
    def forward(self,x):
        return self.Net(x)

In [45]:
model=Reg()
model.train()
optimizer=paddle.optimizer.SGD(learning_rate=0.01,parameters=model.parameters())
epochs=10
batch_size=16
for epoch in range(epochs):
    np.random.shuffle(training_data)
    mini_batches=[training_data[i:i+batch_size] for i in range(0,len(training_data),batch_size)]
    
    for i,mini_batch in enumerate(mini_batches):
        x=np.array(mini_batch[:,:-1])
        y=np.array(mini_batch[:,-1])
        x=paddle.to_tensor(x)
        y=paddle.to_tensor(y)
        y_pre=model(x)
        loss=F.square_error_cost(y_pre,label=y)  #两个向量之间损失    长度等于batch_size 
        avg_loss=paddle.mean(loss)
        if i%10==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch,i, avg_loss.numpy()))
        avg_loss.backward()  #用这个算吗？ 
        optimizer.step()
        optimizer.clear_grad()   #不一样

epoch: 0, iter: 0, loss is: [1.1948652]
epoch: 0, iter: 10, loss is: [0.28370228]
epoch: 0, iter: 20, loss is: [0.12305724]
epoch: 1, iter: 0, loss is: [0.09070122]
epoch: 1, iter: 10, loss is: [0.08819264]
epoch: 1, iter: 20, loss is: [0.05256382]
epoch: 2, iter: 0, loss is: [0.05866911]
epoch: 2, iter: 10, loss is: [0.12725358]
epoch: 2, iter: 20, loss is: [0.02419819]
epoch: 3, iter: 0, loss is: [0.09099519]
epoch: 3, iter: 10, loss is: [0.10967546]
epoch: 3, iter: 20, loss is: [0.01771724]
epoch: 4, iter: 0, loss is: [0.08686951]
epoch: 4, iter: 10, loss is: [0.07093805]
epoch: 4, iter: 20, loss is: [0.11444576]
epoch: 5, iter: 0, loss is: [0.03867127]
epoch: 5, iter: 10, loss is: [0.03514868]
epoch: 5, iter: 20, loss is: [0.04154563]
epoch: 6, iter: 0, loss is: [0.02482323]
epoch: 6, iter: 10, loss is: [0.03945965]
epoch: 6, iter: 20, loss is: [0.09589551]
epoch: 7, iter: 0, loss is: [0.0488534]
epoch: 7, iter: 10, loss is: [0.04810118]
epoch: 7, iter: 20, loss is: [0.05040831]
ep

In [28]:
x=np.ones(2,10)
print(x[:,-1])

In [46]:
paddle.save(model.state_dict(),'lr_pdparams')